In [23]:
import numpy as np
import cvxopt
import struct
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches
from matplotlib.patches import Polygon
import matplotlib.cm as cmx
import matplotlib.colors as colors

### Control_horizon=1, planning_horizon=1

In [33]:
del_t = 0.1
v_init = 10
w_init = 0.1
a_max = 10
alpha_min = -0.5
alpha_max = 0.5

class agent:
    def __init__(self,rad,start,goal,theta,vmax,wmax):
        self.rad = rad
        self.xpath = []
        self.ypath = []
        self.vplot = []
        self.wplot = []
        self.time = 0
        self.goal = goal
        self.theta = theta
        self.v = v_init
        self.w = w_init
        self.pos = start
        self.vmax = vmax
        self.wmax = wmax
        
    def __str__(self):
        return (str(self.time)+" "+str(self.theta)+" "+str(self.v)+" "+str(self.w)+" "+str(self.pos))    
            

    def opt_traj(self):
        self.visualize_traj()
        print ("start v: ",self.v,"start w: ",self.w)
        while (np.linalg.norm(self.pos-self.goal)>0.7):
            x_new = self.pos[0] + self.v*del_t*np.cos(self.theta+self.w*del_t)
            y_new = self.pos[1] + self.v*del_t*np.sin(self.theta+self.w*del_t)
            self.theta += self.w*del_t
            self.xpath.append(x_new)
            self.ypath.append(y_new)
            self.pos = [x_new,y_new]
            theta_new = self.theta
            v_new = self.v
            w_new = self.w
            v_orig = self.v
            w_orig = self.w
            diff_v = 999
            diff_w = 999
#             xgi = self.pos[0]
#             ygi = self.pos[1]
            tol = 1e-1
            while (np.linalg.norm(np.array([diff_w,diff_v])**2)>tol):
                # vi = self.v, wi = self.w
#                 theta_new += self.w*del_t
#                 xgi += (self.v*del_t*np.cos(theta_new) - self.goal[0])
#                 ygi += (self.v*del_t*np.sin(theta_new) - self.goal[1])
                w_guess = self.w
                v_guess = self.v
                x = del_t*np.cos(self.theta+w_guess*del_t)
                a = -v_guess*del_t**2*np.sin(self.theta+w_guess*del_t)
                y = del_t*np.sin(self.theta+w_guess*del_t)
                b = v_guess*del_t**2*np.cos(self.theta+w_guess*del_t)
                kx = self.pos[0]+v_guess*del_t**2*w_guess*np.sin(self.theta+w_guess*del_t) - self.goal[0]
                ky = self.pos[1]-v_guess*del_t**2*w_guess*np.cos(self.theta+w_guess*del_t) - self.goal[1]
                arrx = np.array([[x],[a]])
                arry = np.array([[y],[b]])
                p_mat = 2*(np.matmul(arrx,arrx.T)+np.matmul(arry,arry.T))
                P = cvxopt.matrix(p_mat,tc='d')
                q_mat = 2*(kx*arrx+ky*arry)
                Q = cvxopt.matrix(q_mat,tc='d')
                g_mat = np.array([[0,-1],[-1,0],[1,0],[0,1],[1,0],[-1,0],[0,1],[0,-1]])
                h = cvxopt.matrix(np.array([self.wmax,0,self.vmax,self.wmax,v_guess+5,-(v_guess-5),w_guess+0.1,-(w_guess-0.1)]),tc='d')
#                 print (arrx,arry,kx,ky)
                g = cvxopt.matrix(g_mat,tc='d')
                sol = cvxopt.solvers.qp(P,Q,g,h,options={'show_progress': False})
                v_new = sol['x'][0] #vd
                w_new = sol['x'][1] #wd
#                 print ("Optimizer op:",v_new,w_new)
#                 print (v_new,w_new)
                diff_v = v_new - self.v
                diff_w = w_new - self.w
                print (diff_v,diff_w)
                self.v = v_new #vd+vi
                self.w = w_new #wd+wi
            print ("#####################End of Optimization######################")
#             print ("new v: ",self.v,"new w: ",self.w)
#             print ("Opti complete")
#             self.vplot.append(self.v)
#             self.wplot.append(self.w)
            print (self)
            self.time+=1
            self.visualize_traj()

             
    def visualize_traj(self):
        figure = plt.figure()
#         print (self.pos)
        print ("Theta: ",self.theta)
        ax = figure.add_subplot(1,1,1)
        robot = matplotlib.patches.Rectangle(
            (self.pos[0]-self.rad*np.sqrt(2)*np.cos(self.theta+np.pi/4),self.pos[1]-self.rad*np.sqrt(2)*np.sin(self.theta+np.pi/4)),
            height = self.rad*2,
            width = self.rad*2,
            angle = self.theta*180/np.pi,
            edgecolor='black',
            linewidth=1.0,
            animated=True,
            alpha=1,
            zorder=2)
        ax.add_patch(robot)
        name = 'data/snap%s.png'%str(self.time)
        ax.plot([self.goal[0]], [self.goal[1]], '*', color="red", markersize =15,linewidth=3.0)
        ax.plot(self.xpath,self.ypath,'b-')
        ax.set_aspect('equal')
        ax.set_xlim(-10.0, 100.0)
        ax.set_ylim(-10.0, 100.0)
        ax.set_xlabel(r'$x (m)$')
        ax.set_ylabel(r'$y (m)$')
        ax.grid(True)
        plt.savefig(name, dpi = 200)
        plt.cla()
        plt.close(figure)
        return figure
    


In [37]:
bot = agent(2,np.array([0,0]),np.array([80,50]),np.pi/2,20,0.5)
bot.opt_traj()

Theta:  1.5707963267948966
start v:  10 start w:  0.1
5.000000003192696 -0.0999999122265636
4.999999627382866 -0.09999988231174561
3.3631284424018304e-08 -0.09999982092591811
#####################End of Optimization######################
0 1.5807963267948966 19.999999664206847 -0.19999961546422731 [-0.009999833334166612, 0.9999500004166653]
Theta:  1.5807963267948966
5.040142525558622e-08 -0.09999984772272777
#####################End of Optimization######################
1 1.5607963652484738 19.999999714608272 -0.2999994631869551 [0.009999756095071479, 2.9998499684414166]
Theta:  1.5607963652484738
6.19865474504877e-08 -0.09999994964590159
#####################End of Optimization######################
2 1.5307964189297782 19.99999977659482 -0.39999941283285667 [0.08997823920471247, 4.998250160615845]
Theta:  1.5307964189297782
-1.343707360490498e-06 -0.0999999925232471
#####################End of Optimization######################
3 1.4907964776464926 19.99999843288746 -0.4999994053561

-1.1324348747621116e-07 0.09999976710783864
#####################End of Optimization######################
36 0.12076620352567705 19.99999985062943 0.2999503104306263 [47.34284425027735, 43.80217132166577]
Theta:  0.12076620352567705
-5.56467377066383e-07 0.0999881305749592
#####################End of Optimization######################
37 0.1507612345687397 19.999999294162052 0.3999384410055855 [49.320158303488, 44.102552871078245]
Theta:  0.1507612345687397
5.474297246621518e-07 -0.09993471129898795
#####################End of Optimization######################
38 0.19075507866929825 19.999999841591777 0.30000372970659756 [51.28388093791634, 44.481753520733086]
Theta:  0.19075507866929825
-2.0661394017906787e-07 -0.0999901855308131
#####################End of Optimization######################
39 0.22075545163995802 19.999999634977836 0.20001354417578446 [53.235345540347566, 44.91968712539781]
Theta:  0.22075545163995802
-5.213818248250845e-07 -0.09999418241871642
####################

### Control_horizon = 1, planning_horizon = p

In [67]:
del_t = 0.1
planning_horizon = 10
v_guess = 20*np.ones((planning_horizon))
w_guess = 0.1*np.ones((planning_horizon))
v_init = v_guess[0]
w_init = w_guess[0]
a_max = 10
alpha_min = -0.5
alpha_max = 0.5

class mpc_agent:
    def __init__(self,rad,start,goal,theta,vmax,wmax):
        self.rad = rad
        self.xpath = []
        self.ypath = []
        self.vplot = []
        self.wplot = []
        self.time = 0
        self.goal = goal
        self.theta = theta
        self.v = v_init
        self.w = w_init
        self.v_guess = v_guess
        self.w_guess = w_guess
        self.prev_v = v_guess
        self.prev_w = w_guess
        self.pos = start
        self.vmax = vmax
        self.wmax = wmax
        
    def __str__(self):
        return (str(self.time)+" "+str(self.theta)+" "+str(self.v)+" "+str(self.w)+" "+str(self.pos))    
            
    def p_constructor(self):
#         print (self.w_guess)
        wg = np.cumsum(self.w_guess)
        p_theta = self.theta+wg*del_t
        
        xs = del_t*np.cos(p_theta)
        qx = np.sin(p_theta)
        bs = -self.v_guess*del_t**2*qx
        brev = bs[::-1]
        dterms = np.cumsum(brev)
        dterms = dterms[::-1]
        arrx = np.concatenate((xs, dterms), axis=None)
        ksx = bs*self.w_guess
        kx = np.sum(ksx) + self.pos[0] - self.goal[0]
        
        ys = del_t*np.sin(p_theta)
        qy = np.cos(p_theta)
        by = self.v_guess*del_t**2*qy
        byrev = by[::-1]
        dyterms = np.cumsum(byrev)
        dyterms = dyterms[::-1]
        arry = np.concatenate((ys, dyterms), axis=None)
        ksy = by*self.w_guess
        ky = np.sum(ksy) + self.pos[1] - self.goal[1]
        
        
        return arrx,arry,kx,ky
        
    def opt_traj(self):
        self.visualize_traj()
        print ("start v: ",self.v,"start w: ",self.w)
        
        while (np.linalg.norm(self.pos-self.goal)>0.7):
            x_new = self.pos[0] + self.v*del_t*np.cos(self.theta+self.w*del_t)
            y_new = self.pos[1] + self.v*del_t*np.sin(self.theta+self.w*del_t)
            self.theta += self.w*del_t
            self.xpath.append(x_new)
            self.ypath.append(y_new)
            self.pos = [x_new,y_new]
            theta_new = self.theta
            v_new = self.v
            w_new = self.w
            diff_v = 999
            diff_w = 999
            tol = 1e-1
            
            while (np.linalg.norm(np.array([diff_w,diff_v])**2)>tol):
                arrx,arry,kx,ky = self.p_constructor()
                arrx = arrx.reshape((-1,1))
                arry = arry.reshape((-1,1))
#                 print (arrx.shape,arry.shape,kx.shape,ky.shape)
                p_mat = 2*(np.matmul(arrx,arrx.T)+np.matmul(arry,arry.T))
                P = cvxopt.matrix(p_mat,tc='d')
                q_mat = 2*(kx*arrx+ky*arry)
                Q = cvxopt.matrix(q_mat,tc='d')
                g_mat = np.eye(2*planning_horizon)
                g_mat = np.concatenate((g_mat,-np.eye(2*planning_horizon)),axis=0)
                g_mat = np.concatenate((g_mat,np.eye(2*planning_horizon)),axis=0)
                g_mat = np.concatenate((g_mat,-np.eye(2*planning_horizon)),axis=0)
#                 g_mat[:planning_horizon,:planning_horizon] = np.eye(planning_horizon)
#                 print (g_mat)
#                 g_mat = np.array([[0,-1],[-1,0],[1,0],[0,1]])
#                 h_mat = np.concatenate((self.wmax*np.ones(2*planning_horizon),np.zeros(2*planning_horizon),self.vmax*np.ones(2*planning_horizon),self.wmax*np.ones(2*planning_horizon)),axis=None)
#                 h = cvxopt.matrix(h_mat,tc='d')
#                 h_mat = np.array([self.wmax,0,self.vmax,self.wmax])
                h_mat = np.concatenate((self.vmax*np.ones(planning_horizon),self.wmax*np.ones(planning_horizon),np.zeros(planning_horizon),self.wmax*np.ones(planning_horizon),self.v_guess+5,self.w_guess+0.1,-(self.v_guess-5),-(self.w_guess-0.1)),axis=None)
                h = cvxopt.matrix(h_mat,tc='d')
#                 print ("h_mat:",h_mat)
#                 print ("p_mat:",p_mat)
#                 print (arrx,arry,kx,ky)
                g = cvxopt.matrix(g_mat,tc='d')
#                 print (p_mat.shape,q_mat.shape,g_mat.shape,h_mat.shape)
                sol = cvxopt.solvers.qp(P,Q,g,h,options={'show_progress': False})
                v_new = sol['x'][:planning_horizon] #vd
                w_new = sol['x'][planning_horizon:] #wd
#                 print ("Optimizer op:",v_new,w_new)
                diff_v = v_new[0] - self.v
                diff_w = w_new[0] - self.w
#                 print (diff_v,diff_w)
                self.v = v_new[0]
                self.w = w_new[0]
                self.prev_v = np.array(v_new)
                self.prev_w = np.array(w_new)
#                 print (self.v,self.w)
                self.v_guess = self.prev_v[1:]
                self.v_guess = np.concatenate((self.v_guess,np.array([[self.v]])),axis=0)
                self.v_guess = self.v_guess.reshape((-1,))
                self.w_guess = self.prev_w[1:]
                self.w_guess = np.concatenate((self.w_guess,np.array([[self.w]])),axis=0)
                self.w_guess = self.w_guess.reshape((-1,))
#                 self.v_guess = 10*np.ones((planning_horizon))
#                 self.w_guess = 0.1*np.ones((planning_horizon))
#                 print (self.v_guess,self.w_guess)
                
            print ("##########End of Optimization#############")
            print ("new v: ",self.v,"new w: ",self.w)
            print ("Opti complete")
            print (self)
            self.time+=1
            self.visualize_traj()

             
    def visualize_traj(self):
        figure = plt.figure()
#         print (self.pos)
        print ("Theta: ",self.theta)
        ax = figure.add_subplot(1,1,1)
        robot = matplotlib.patches.Rectangle(
            (self.pos[0]-self.rad*np.sqrt(2)*np.cos(self.theta+np.pi/4),self.pos[1]-self.rad*np.sqrt(2)*np.sin(self.theta+np.pi/4)),
            height = self.rad*2,
            width = self.rad*2,
            angle = self.theta*180/np.pi,
            edgecolor='black',
            linewidth=1.0,
            animated=True,
            alpha=1,
            zorder=2)
        ax.add_patch(robot)
        name = 'data/snap%s.png'%str(self.time)
        ax.plot([self.goal[0]], [self.goal[1]], '*', color="red", markersize =15,linewidth=3.0)
        ax.plot(self.xpath,self.ypath,'b-')
#         ax.plot(self.prev_v,self.ypath,'b-')
        ax.set_aspect('equal')
        ax.set_xlim(-10.0, 100.0)
        ax.set_ylim(-10.0, 100.0)
        ax.set_xlabel(r'$x (m)$')
        ax.set_ylabel(r'$y (m)$')
        ax.grid(True)
        plt.savefig(name, dpi = 200)
        plt.cla()
        plt.close(figure)
        return figure
    


In [69]:
bot1 = mpc_agent(2,np.array([0,0]),np.array([60,90]),0,30,0.5)
bot1.opt_traj()

Theta:  0
start v:  20.0 start w:  0.1
##########End of Optimization#############
new v:  29.999999854148886 new w:  0.4000002649437907
Opti complete
0 0.010000000000000002 29.999999854148886 0.4000002649437907 [1.9999000008333305, 0.019999666668333332]
Theta:  0.010000000000000002
##########End of Optimization#############
new v:  29.999998322453372 new w:  0.49999991285816314
Opti complete
1 0.050000026494379075 29.999998322453372 0.49999991285816314 [4.996150763478843, 0.16993725313522015]
Theta:  0.050000026494379075
##########End of Optimization#############
new v:  29.99999983167985 new w:  0.5000000021266026
Opti complete
2 0.10000001778019539 29.99999983167985 0.5000000021266026 [7.98116308707116, 0.46943753940228583]
Theta:  0.10000001778019539
##########End of Optimization#############
new v:  29.999999805494202 new w:  0.500000002116591
Opti complete
3 0.15000001799285564 29.999999805494202 0.500000002116591 [10.947476296169821, 0.9177519876801841]
Theta:  0.1500000179928556

##########End of Optimization#############
new v:  19.532599604326013 new w:  0.10022072304663474
Opti complete
35 1.5344905253098062 19.532599604326013 0.10022072304663474 [61.113237184435754, 70.48647800932905]
Theta:  1.5344905253098062
##########End of Optimization#############
new v:  17.576495492453805 new w:  0.14306836058315048
Opti complete
36 1.5445125976144698 17.576495492453805 0.14306836058315048 [61.16457022933929, 72.43906331899674]
Theta:  1.5445125976144698
##########End of Optimization#############
new v:  15.824584957944476 new w:  0.12799359731154075
Opti complete
37 1.5588194336727847 15.824584957944476 0.12799359731154075 [61.18562090685584, 74.19658680587781]
Theta:  1.5588194336727847
##########End of Optimization#############
new v:  14.242843944589383 new w:  0.12621502342556615
Opti complete
38 1.5716187934039387 14.242843944589383 0.12621502342556615 [61.18431938772959, 75.77904476644422]
Theta:  1.5716187934039387
##########End of Optimization#############


In [74]:
wg = np.array([[5,6,7]])
print (wg)
c = np.cumsum(wg)
c[::-1]

[[5 6 7]]


array([18, 11,  5])

In [40]:
a = np.array([1,0,3,2])
b = np.array([4,5,6,1])
# np.concatenate((a, b), axis=None)

In [70]:
np.concatenate((np.zeros(2),np.ones(2)),axis=None)

array([0., 0., 1., 1.])

In [71]:
np.array([a,b])

array([[1, 0, 3],
       [4, 5, 6]])

In [87]:
a.reshape((-1,1))

array([[1],
       [0],
       [3]])

In [113]:
c = np.zeros((4,4))

In [114]:
c[:2,:2] = np.eye(2)
c

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [130]:
np.concatenate((c,b),axis = 0)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [4., 5., 6., 1.]])